In [1]:
import glob
import torch
import numpy as np
import torch.nn as nn
import scipy.io as sio
import matplotlib.pyplot as plt
import torch.nn.functional as F
import scipy.ndimage as ndimage
from torchvision import transforms, utils
from toolz.curried import pipe, curry, compose
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import csv
import numpy as np


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import os

import scores

In [3]:
import argparse
import datetime
import numpy as np
import math
import os
import gc

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader



## Reading in the Data

In [4]:
path_f=os.getcwd()

path_f_1=os.path.join(path_f, 'data')


names=[]
for files_txts in os.listdir(path_f_1):
    if files_txts.endswith(".csv"):
        #print(files_txts)
        names.append(files_txts)

print(names)
path_train=os.path.join(path_f_1, names[0])
path_test=os.path.join(path_f_1, names[1])

df_train=pd.read_csv(path_train)
print(df_train.shape)
print(path_train)

['training_data.csv', 'test_data.csv']
(2572, 99)
/Users/davidmontesdeoca/Desktop/challenge_data/data/training_data.csv


## Feature Engineering

In [5]:
stab_vector=df_train['stabilityVec'].values
y=[]
for x in stab_vector:
    #print(x)
    a=np.fromstring(x[1:-1],sep=',').astype(int)
    y.append(a)
y=np.array(y) 

df_tmp = pd.DataFrame(y, columns = ['A', 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B','B'])
stab_vec_list=[ 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B','Stable_compunds']

df_train=df_train.drop("stabilityVec",axis=1) #removing the results which originally are a string
feature_cols=list(df_train)



df_train['formulaA']=df_train['formulaA_elements_Number']
df_train['formulaB']=df_train['formulaB_elements_Number']


df_train=pd.concat([df_train, df_tmp],axis=1)

y_all=df_train[stab_vec_list[0:-1]]


df_tmp_stable = pd.DataFrame( columns = ['Stable_compunds'])
df_tmp_stable['Stable_compunds']=np.logical_not(y_all.sum(axis=1)==0).astype(int) ## A one means it has a stable value  a 0 

df_train=pd.concat([df_train, df_tmp_stable],axis=1)

y_all=df_train[stab_vec_list]
print(df_train.shape)


print(df_train.shape)


df_stable=df_train.loc[np.logical_not(y_all.sum(axis=1)==0)]
print(df_stable.shape)
df_train.head()

(2572, 110)
(2572, 110)
(1228, 110)


,formulaA,formulaB,formulaA_elements_AtomicVolume,formulaB_elements_AtomicVolume,formulaA_elements_AtomicWeight,formulaB_elements_AtomicWeight,formulaA_elements_BoilingT,formulaB_elements_BoilingT,formulaA_elements_BulkModulus,formulaB_elements_BulkModulus,...,A82B,A73B,A64B,A55B,A46B,A37B,A28B,A19B,B,Stable_compunds
0,89,47,37.433086,17.075648,227.0,107.868200,3473.0,2435.0,0.0,100.0,...,0,1,0,1,0,0,0,0,1,1
1,89,13,37.433086,16.594425,227.0,26.981539,3473.0,2792.0,0.0,76.0,...,0,1,0,0,0,0,0,0,1,1
2,89,33,37.433086,21.723966,227.0,74.921600,3473.0,887.0,0.0,22.0,...,0,0,0,0,0,0,0,0,1,0
3,89,56,37.433086,64.969282,227.0,137.327000,3473.0,2143.0,0.0,9.6,...,0,0,0,0,0,0,0,0,1,0
4,89,83,37.433086,35.483459,227.0,208.980400,3473.0,1837.0,0.0,31.0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
#X_train_new=df_stable[feature_cols] #training only on stable elements
#y_target=df_stable[stab_vec_list]
###

X_train_new=df_train[feature_cols]   #training  on all elements
y_target=df_train[stab_vec_list]

print(X_train_new.shape,y_target.shape)

(2572, 98) (2572, 10)


In [8]:
# Normalizing such that the magnitude is one
from sklearn.preprocessing import normalize

X_train_new_mag_1=normalize(X_train_new, axis=1) # vector magnitude is one
X_train_new_mag_1=pd.DataFrame(data=X_train_new_mag_1,columns=feature_cols)
print(X_train_new_mag_1.shape)


## Normalizing by Zscore
from scipy.stats import zscore
X_train_new_Z_score=X_train_new.apply(zscore)
print(X_train_new_Z_score.shape)



## Normalizing so that range is 0-1
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_new_0_1=min_max_scaler.fit_transform(X_train_new)
X_train_new_0_1=pd.DataFrame(data=X_train_new_0_1,columns=feature_cols)
print(X_train_new_0_1.shape)


## Normalizing so that range is -1 to 1
from sklearn import preprocessing
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train_new_m1_p1=max_abs_scaler.fit_transform(X_train_new)
X_train_new_m1_p1=pd.DataFrame(data=X_train_new_m1_p1,columns=feature_cols)
print(X_train_new_m1_p1.shape)


# Using PCA as input
X_train_4_PCA=df_train[feature_cols]
indx_4_PC=X_train_4_PCA.index
X_train_new_mag_1_PCA=normalize(X_train_4_PCA, axis=1)


pca = PCA()
pca.fit(X_train_new_mag_1_PCA)
components = pca.components_[:20,:]
new_data = np.dot(X_train_new_mag_1_PCA, components.T)
X_train_new_PCA=new_data

print(X_train_new_PCA.shape)


## Taking Logarithm of High Values

X_train_new_log=X_train_new.copy()
X_train_new_log[X_train_new_log>100]=X_train_new_log[X_train_new_log>100].apply(np.log)
print(X_train_new_log.shape)

(2572, 98)
(2572, 98)
(2572, 98)
(2572, 98)
(2572, 20)
(2572, 98)


/anaconda3/envs/pytorch/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [9]:
device = torch.device("cpu")#device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(98, 60),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(60, 20),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(20, 10),
            nn.ReLU())

    def forward(self, batch):
        return self.layers(batch)
    

In [26]:
model = Network().to(device)

In [27]:
class feature_Dataset(Dataset):
    """Face landmarks Dataset"""
    
    def __init__(self, X_features, y_target):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """        
        self.X_features=X_features
        self.y_target=y_target
        assert len(X_features) == len(y_target)
        


        self.X_features = torch.from_numpy(X_features.values).float()
        self.y_target = torch.from_numpy(y_target.values).float()
        
    def __len__(self):
        return len(self.X_features)

    def __getitem__(self, idx):
        
        
        item = {'index': idx}
        item['features'] = self.X_features[idx,:]
        item['outputs'] = self.y_target[idx,:]
        return item


In [28]:
## test-train split
X_train, X_test, y_train, y_test = train_test_split(X_train_new_m1_p1, y_target,
                                                    test_size=.15,
                                                    shuffle=True,
                                                    random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)
#X_train.head()

(2186, 98) (2186, 10)
(386, 98) (386, 10)


In [29]:
X_train.head()

,formulaA,formulaB,formulaA_elements_AtomicVolume,formulaB_elements_AtomicVolume,formulaA_elements_AtomicWeight,formulaB_elements_AtomicWeight,formulaA_elements_BoilingT,formulaB_elements_BoilingT,formulaA_elements_BulkModulus,formulaB_elements_BulkModulus,...,formulaA_elements_Row,formulaB_elements_Row,formulaA_elements_ShearModulus,formulaB_elements_ShearModulus,formulaA_elements_SpaceGroupNumber,formulaB_elements_SpaceGroupNumber,avg_coordination_A,avg_coordination_B,avg_nearest_neighbor_distance_A,avg_nearest_neighbor_distance_B
1711,0.838710,0.215054,0.000413,0.001153,0.819581,0.168375,0.698245,0.299370,0.605263,0.044737,...,0.857143,0.571429,0.274775,0.033333,0.982533,0.982533,1.000000,1.000000,0.528179,0.731902
332,0.376344,0.064516,0.001142,0.000237,0.335690,0.050459,0.056568,0.732663,0.005000,0.086842,...,0.571429,0.285714,0.000000,0.000000,0.279476,0.847162,0.083333,0.250000,0.448680,0.267487
432,0.516129,0.301075,0.000580,0.000294,0.472258,0.246581,0.177202,0.542852,0.110526,0.473684,...,0.714286,0.571429,0.085586,0.342342,0.847162,0.982533,0.500000,1.000000,0.564966,0.465925
407,0.215054,0.483871,0.001153,0.000369,0.168375,0.432324,0.299370,0.676095,0.044737,1.000000,...,0.571429,0.714286,0.033333,0.675676,0.982533,0.982533,1.000000,1.000000,0.731902,0.511166
508,0.258065,0.053763,0.000325,0.000196,0.218444,0.045419,0.501619,0.728063,0.421053,0.842105,...,0.571429,0.285714,0.518018,0.000000,1.000000,0.724891,0.666667,0.458333,0.463537,0.324075


In [30]:
batch_size = 64
train_dataset = feature_Dataset(X_train, y_train) ## Now you can feed any X-train and Y-train. normalize it and feed the highest pearson correlated one

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6)

val_dataset = feature_Dataset(X_test, y_test)

valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=6)

In [31]:
item = train_dataset[0]

x = item['features'][None]
print("features: ", x)
y = item['outputs']
print("outputs: ", y)


features:  tensor([[ 0.8387,  0.2151,  0.0004,  0.0012,  0.8196,  0.1684,  0.6982,  0.2994,
          0.6053,  0.0447,  0.5556,  0.1111,  0.5574,  0.7213,  0.9336,  0.0686,
          0.9622,  0.4919,  0.7600,  0.3333,  0.3644,  0.2486,  0.0000,  0.0000,
         -0.4652, -0.1503,  0.5062,  0.6884,  0.5062,  0.6884,  0.0000,  0.0000,
          0.1297,  0.3263,  0.5612,  0.3980,  1.0000,  0.1648,  0.0259,  0.1261,
          0.3417,  0.3426,  0.1889,  0.0727,  0.1622,  0.4747,  0.0000,  1.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.5340,  0.2917,  0.6176,  0.0686,  0.5265,  0.7614,
          0.0909,  0.0000,  0.8276,  0.0690,  0.1111,  0.0000,  0.9000,  0.0000,
          0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  0.5000,  1.0000,  0.8387,  0.2151,  0.1094,  0.4331,
          0.8571,  0.5714,  0.2748,  0.0333,  0.9825,  0.9825,  1.0000,  1.0000,
          0.5282,

In [32]:
y.shape

torch.Size([10])

# Training

In [33]:
def mse_loss(y1, y2):
    """standard MSE definition"""
    return ((y1 - y2) ** 2).sum() / y1.data.nelement() * 500


def class_loss(y1, y2):
    """Standard MAE definition"""
    return ((y1 - y2).abs()).sum() 

In [34]:
# Loss and optimizer
num_epochs = 20
learning_rate = .00001
#criterion = nn.CrossEntropyLoss()
criterion = class_loss
#criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_step = len(trainloader)
print("No. of steps per epoch: %d" % total_step)

No. of steps per epoch: 35


In [35]:
print(criterion)
print(optimizer)
for epoch in range(10):
    if epoch > 0:
        learning_rate = .00001
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif epoch >= 10:
        learning_rate = 0.0002
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif epoch > 80:
        learning_rate = 0.0001
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for i, item in enumerate(trainloader):
        model.train()
        x = item['features'].to(device)
        target = item['outputs'].to(device)
        # Forward pass
        output = model(x)
        output1=Variable(data=(output>.9).float(), requires_grad=True)
        loss = criterion(output1, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 35 == 0:
            niter = 5
            val_loss=0
            for i1, item1 in enumerate(valloader):
                x_val = item1['features'].to(device)
                y_val = item1['outputs'].to(device)
                model.eval()
                y_pre = model(x_val)
                y_pre1=Variable(data=(y_pre>.9).float(), requires_grad=True)
                val_loss += criterion(y_pre1, y_val)

                if i1 == niter-1:
                    break
                if val_loss.item()/niter < 6.5:
                    torch.save(model.state_dict(), "learnt_weights_C0_%d_%d" % (i, epoch))
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.6f}, Val Loss: {:.6f}' 
                   .format(epoch+1, num_epochs, i, total_step, loss.item(), val_loss.item()/niter))
        
        

<function class_loss at 0x1a1e082c80>
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0
)
Epoch [1/20], Step [0/35], Loss: 108.000000, Val Loss: 17.600000
Epoch [1/20], Step [0/35], Loss: 108.000000, Val Loss: 33.400000
Epoch [1/20], Step [0/35], Loss: 108.000000, Val Loss: 50.600000
Epoch [1/20], Step [0/35], Loss: 108.000000, Val Loss: 70.200000
Epoch [2/20], Step [0/35], Loss: 108.000000, Val Loss: 15.000000
Epoch [2/20], Step [0/35], Loss: 108.000000, Val Loss: 38.800000
Epoch [2/20], Step [0/35], Loss: 108.000000, Val Loss: 57.200000
Epoch [2/20], Step [0/35], Loss: 108.000000, Val Loss: 71.200000
Epoch [3/20], Step [0/35], Loss: 91.000000, Val Loss: 13.400000
Epoch [3/20], Step [0/35], Loss: 91.000000, Val Loss: 30.400000
Epoch [3/20], Step [0/35], Loss: 91.000000, Val Loss: 50.400000
Epoch [3/20], Step [0/35], Loss: 91.000000, Val Loss: 67.000000
Epoch [4/20], Step [0/35], Loss: 76.000000, Val Loss: 17.000000
Epo

In [36]:
y_pre1[:5]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<SliceBackward>)

In [37]:
y_val[:5]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]])

In [ ]:
output1[:1]

In [ ]:
target[:1]